In [3]:
import pandas as pd
import numpy as np
import re
import json


from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [4]:
DATA_IN_PATH='./data_in2/'
train_data=pd.read_csv(DATA_IN_PATH+'train.csv',encoding='utf-8')